# Learn ANN

In [0]:
# learning ANN file from the matlab
def learn_ann(inp, out, w,b,params):
  a = params.N_layers -1
  iterations =len(inp)
  for its in range(iterations):
    x = [[] for i in range(params.N_layers)]
    d = [[] for i in range(params.N_layers)]
    grad_b = [[] for i in range(params.N_layers)]
    grad_w = [[] for i in range(params.N_layers)]
    # start organising the data properly into these silly cell arrays
    x[0] = inp[:,its]
    x_out = out[:,its]
    # make a prediction
    for i in range(1,params.N_layers):
      x[i] = np.dot(w[i-1], params.act_fn) + b[i-1]

    # backprop errors
    d[params.N_layers] = x_out - x[params.N_layers]
    for i in reversed(range(1,params.N_layers-1)):
      d[i] = np.dor(w[i].T, d[i+1]) * params.act_fn_deriv(x[i])
    
    # find gradients
    for i in range(a):
      grad_b[i] = d[i+1]
      grad_w[i] = np.dot(d[i+1], params.act_fn(x[i]).T) - (params.weight_decay_rate * w[i])

    # update weights
    for i in range(a):
      w[i] = w[i] + params.learning_rate * grad_w[i]
      b[i] = b[i] + params.learning_rate * grad_b[i]

  return w,b

# Learn PC

In [0]:
def learn_pc(inp, out, w,b,params):
  v_out = params.var[-1]
  a = params.N_layers -1
  iterations = len(inp)
  for its in range(iterations):
    x = [[] for i in range(params.N_layers)]
    grad_b = [[] for i in range(params.N_layers)]
    grad_w = [[] for i in range(params.N_layers)]
    #organize data into cell arrays
    x[0] = inp[:, its]
    x_out = out[:,its]
    #make prediciton
    for i in range(1,params.N_layers):
      x[i] = np.dot(w[i-1], params.act_fn(x[i-1])) + b[i-1]
    
    #infer
    x[params.N_layers] = x_out
    x,es,i = infer_pc(x,w,b,params)
    #calculate gradients
    for i in range(a):
      grad_b[i] = v_out * e[i+1]
      grad_w[i] = v_out * np.dot(e[i+1], params.act_fn(x[i]).T) - params.weight_decay_rate * w[i]
    
    #update weights
    for i in range(a):
      w[i] = w[i]+ params.learning_rate * grad_w[i]
      b[i] = b[i] + params.learning_rate * grad_b[i]

  return w,b


# Infer PC

In [0]:
def infer_pc(x,w,b,params):
  # initialize prediction errors
  es = [np.zeros([n,1]) for n in params.neurons]
  act_fn = params.act_fn
  act_fn_deriv = params.act_fn_deriv
  # calculate initial layers
  for i in range(1,params.N_layers):
    e[i] = (x[i] - np.dot(w[i-1], act_fn(x[i-1])) - b[i-1]) / params.var[i]
  
  #update variable nodes
  for j in range(params.n_iterations):
    for i in range(1,params.N_layers-1):
      g = np.dot(w[i].T * e[i+1]) * act_fn_deriv(x[i])
      x[i] = x[i] + params.beta * (-e[i] + g)
    
    # calculate errors
    for i in range(1,params.N-layers):
      e[i] = (x[i] - np.dot(w[i-1], act_fn(x[i-1]) - b[i]))/var[i]

  return x, es, i



# Initialization

In [0]:
def w_init(params):
  w = [[] for i in range(params.N_layers)]
  b = [[] for i in range(params.N_layers)]
  neurons = params.neurons
  
  for i in range(params.N_layers-1):
    norm_b = 0
    if params.act_fn_type == "linear":
      norm_w = np.sqrt(1/(neurons[i+1] + neurons[i]))
    if params.act_fn_type =='tanh':
      norm_w = np.sqrt(6/(neurons[i+1] + neurons[i]))
    if params.act_fn_type == 'sigmoid':
      norm_w = 4 * np.sqrt(6/(neurons[i+1] + neurons[i]))
    if params.act_fn_type =='relu':
      norm_w = np.sqrt(1/(neurons[i+1] + neurons[i]))
      norm_b = 0.1
    w[i] = np.random.uniform(low=-1, high=1, size=(neurons[i+1], neurons[i])) * norm_w
    b[i] = np.ones((neurons[i+1],neurons[i])) * norm_b
  return w,b


# Test

In [0]:
def test(inp, out, w,b,params):
  iterations = len(inp)
  x_output = np.zeros(out.shape)
  x = [[] for i in range(params.N_layers)]
  for its in range(iterations):
    #make prediction
    x[0] = inp[:,its]
    for i in range(1,params.N_layers):
      x[i] = np.dot(w[i-1], params.act_fn(x[i-1])) + b[i-1]
    x_output[:,its] = x[params.N_layers]
  return rmse(out, x_output)


# Activation Functions

In [0]:
def linear(xs):
  return xs

def linearderiv(xs):
  return np.ones((len(xs),1))

def tanh(xs):
  return np.tanh(xs)
  
def tanhderiv(xs):
  return 1 - np.tanh(xs) **2

def relu(xs):
    return np.maximum(xs,0,xs)

def reluderiv(xs):
    rel = relu(xs)
    rel[rel>0] = 1
    return rel

def rmse(y,f):
  return np.sqrt(np.mean((y.reshape((-1)) - f.reshape((-1)))**2 ))


In [44]:
# whittington 2017 code copying/transpiling manually into python!
# I'm slowly getting this done, and although it looks SUPER simple, looks like to be good for getting the hang of whittingtons code. That was quite a boring hour spent doing this...
#but nearly done to be honest... so that's good

import numpy as np

class Params(object):
  def __init__(self):
    self.act_fn = tanh
    self.act_fn_type='tanh'
    self.learning_rate = 0.2
    self.n_iterations = 100
    self.epochs = 500
    self.weight_decay_rate = 0
    self.beta = 0.2
    self.neurons = [2,5,1]
    self.N_layers = len(neurons)
    self.var = np.ones([1, N_layers])
    self.var[-1] = 10


params = Params()

sin = np.array([[0,0,1,1],[0,1,0,1]])
sout = np.array([1,0,0,1])


plot_every = 50
run_num = 4

for run in range(run_num):
  w_pc, b_pc = w_init(params)
  w_ann = w_pc
  b_ann = b_pc
  counter = 1
  error_pc = test(sin,sout, w_pc, b_pc, params)
  error_ann = test(sin, sout, w_ann, b_ann, params)


ValueError: ignored

In [43]:
t = params.act_fn
print(t)
print(t(5))

<bound method tanh of <__main__.Params object at 0x7f46ea33d908>>


TypeError: ignored